# Hitchhiker's Guide to Delta Lake (Python)

This tutorial has been adapted for more clarity from its original counterpart [here](https://docs.delta.io/latest/quick-start.html). This notebook helps you quickly explore the main features of [Delta Lake](https://github.com/delta-io/delta). It provides code snippets that show how to read from and write to Delta Lake tables from interactive, batch, and streaming queries.

Here's what we will cover:
* Create a table
* Understanding meta-data
* Read data
* Update table data
* Overwrite table data
* Conditional update without overwrite
* Read older versions of data using Time Travel
* Write a stream of data to a table
* Read a stream of changes from a table

## Configuration
Make sure you modify this as appropriate.

In [1]:
import random

session_id = random.randint(0,1000000)
delta_table_path = "/delta/delta-table-{0}".format(session_id)

delta_table_path

StatementMeta(, , , SessionStarting, )

## Create a table
To create a Delta Lake table, write a DataFrame out in the **delta** format. You can use existing Spark SQL code and change the format from parquet, csv, json, and so on, to delta.

These operations create a new Delta Lake table using the schema that was inferred from your DataFrame. For the full set of options available when you create a new Delta Lake table, see Create a table and Write to a table (subsequent cells in this notebook).

In [2]:
data = spark.range(0,5)
data.show()
data.write.format("delta").save(delta_table_path)

StatementMeta(small, 8, 4, Finished, Available)

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+

## Understanding Meta-data

In Delta Lake, meta-data is no different from data i.e., it is stored next to the data. Therefore, an interesting side-effect here is that you can peek into meta-data using regular Spark APIs. 

In [3]:
[log_line.value for log_line in spark.read.text(delta_table_path + "/_delta_log/").collect()]

StatementMeta(small, 8, 5, Finished, Available)

['{"commitInfo":{"timestamp":1614058299692,"operation":"WRITE","operationParameters":{"mode":"ErrorIfExists","partitionBy":"[]"},"isBlindAppend":true,"operationMetrics":{"numFiles":"6","numOutputBytes":"2407","numOutputRows":"5"}}}', '{"protocol":{"minReaderVersion":1,"minWriterVersion":2}}', '{"metaData":{"id":"d12e1f0b-a3f5-4ebd-840b-dc7b4c4b46a1","format":{"provider":"parquet","options":{}},"schemaString":"{\\"type\\":\\"struct\\",\\"fields\\":[{\\"name\\":\\"id\\",\\"type\\":\\"long\\",\\"nullable\\":true,\\"metadata\\":{}}]}","partitionColumns":[],"configuration":{},"createdTime":1614058292359}}', '{"add":{"path":"part-00000-34c67853-75ca-44a6-afdc-86a242188b51-c000.snappy.parquet","partitionValues":{},"size":262,"modificationTime":1614058299000,"dataChange":true}}', '{"add":{"path":"part-00001-a850ac70-058b-4b39-9ca2-ba25586e8b5c-c000.snappy.parquet","partitionValues":{},"size":429,"modificationTime":1614058297000,"dataChange":true}}', '{"add":{"path":"part-00003-7cc9fc73-04cb-47

## Read data

You read data in your Delta Lake table by specifying the path to the files.

In [4]:
df = spark.read.format("delta").load(delta_table_path)
df.show()

StatementMeta(small, 8, 6, Finished, Available)

+---+
| id|
+---+
|  4|
|  0|
|  1|
|  3|
|  2|
+---+

## Update table data

Delta Lake supports several operations to modify tables using standard DataFrame APIs. This example runs a batch job to overwrite the data in the table.


In [5]:
data = spark.range(5,10)
data.write.format("delta").mode("overwrite").save(delta_table_path)
df.show()

StatementMeta(small, 8, 7, Finished, Available)

+---+
| id|
+---+
|  6|
|  5|
|  9|
|  8|
|  7|
+---+

When you now inspect the meta-data, what you will notice is that the original data is over-written. Well, not in a true sense but appropriate entries are added to Delta's transaction log so it can provide an "illusion" that the original data was deleted. We can verify this by re-inspecting the meta-data. You will see several entries indicating reference removal to the original data.

In [6]:
[log_line.value for log_line in spark.read.text(delta_table_path + "/_delta_log/").collect()]

StatementMeta(small, 8, 8, Finished, Available)

['{"commitInfo":{"timestamp":1614058320094,"operation":"WRITE","operationParameters":{"mode":"Overwrite","partitionBy":"[]"},"readVersion":0,"isBlindAppend":false,"operationMetrics":{"numFiles":"6","numOutputBytes":"2407","numOutputRows":"5"}}}', '{"add":{"path":"part-00000-bb518901-f07c-434c-9140-83798465da11-c000.snappy.parquet","partitionValues":{},"size":262,"modificationTime":1614058319000,"dataChange":true}}', '{"add":{"path":"part-00001-8c1ef250-dbab-4bbb-857a-26aa9f05c421-c000.snappy.parquet","partitionValues":{},"size":429,"modificationTime":1614058319000,"dataChange":true}}', '{"add":{"path":"part-00003-fa61da74-e9c6-4028-90ff-f75a1a8a553d-c000.snappy.parquet","partitionValues":{},"size":429,"modificationTime":1614058319000,"dataChange":true}}', '{"add":{"path":"part-00004-044398cd-b135-4581-a490-22beba084210-c000.snappy.parquet","partitionValues":{},"size":429,"modificationTime":1614058319000,"dataChange":true}}', '{"add":{"path":"part-00006-1557c4bb-8e50-489f-8801-3cd9d9d5d

## Save as catalog tables

Delta Lake can write to managed or external catalog tables.

In [7]:
# Write data to a new managed catalog table.
data.write.format("delta").saveAsTable("ManagedDeltaTable")

StatementMeta(small, 8, 9, Finished, Available)

In [8]:
# Define an external catalog table that points to the existing Delta Lake data in storage.
spark.sql("CREATE TABLE ExternalDeltaTable USING DELTA LOCATION '{0}'".format(delta_table_path))

StatementMeta(small, 8, 10, Finished, Available)

DataFrame[]

In [9]:
# List the 2 new tables.
spark.sql("SHOW TABLES").show()

# Explore their properties.
spark.sql("DESCRIBE EXTENDED ManagedDeltaTable").show(truncate=False)
spark.sql("DESCRIBE EXTENDED ExternalDeltaTable").show(truncate=False)

StatementMeta(small, 8, 11, Finished, Available)

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
| default|  externaldeltatable|      false|
| default|   manageddeltatable|      false|
| default|            nyc_taxi|      false|
| default|nyc_taxi_synapse_...|      false|
| default|       yourtablename|      false|
+--------+--------------------+-----------+

+----------------------------+------------------------------------------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                                                   |comment|
+----------------------------+------------------------------------------------------------------------------------------------------------+-------+
|id                          |bigint                                                                                                      |null   |
|          

## Conditional update without overwrite

Delta Lake provides programmatic APIs to conditional update, delete, and merge (upsert) data into tables. For more information on these operations, see [Table Deletes, Updates, and Merges](https://docs.delta.io/latest/delta-update.html).

In [10]:
from delta.tables import *
from pyspark.sql.functions import *

delta_table = DeltaTable.forPath(spark, delta_table_path)

StatementMeta(small, 8, 12, Finished, Available)

In [11]:
# Update every even value by adding 100 to it
delta_table.update(
  condition = expr("id % 2 == 0"),
  set = { "id": expr("id + 100") })
delta_table.toDF().show()

StatementMeta(small, 8, 13, Finished, Available)

+---+
| id|
+---+
|  5|
|  9|
|106|
|108|
|  7|
+---+

In [12]:
# Delete every even value
delta_table.delete("id % 2 == 0")
delta_table.toDF().show()

StatementMeta(small, 8, 14, Finished, Available)

+---+
| id|
+---+
|  5|
|  9|
|  7|
+---+

In [13]:
# Upsert (merge) new data
new_data = spark.range(0,20).alias("newData")

delta_table.alias("oldData")\
    .merge(new_data.alias("newData"), "oldData.id = newData.id")\
    .whenMatchedUpdate(set = { "id": lit("-1")})\
    .whenNotMatchedInsert(values = { "id": col("newData.id") })\
    .execute()

delta_table.toDF().show(100)

StatementMeta(small, 8, 15, Finished, Available)

+---+
| id|
+---+
| 12|
| -1|
| 16|
|  2|
|  0|
| 11|
| 18|
|  6|
| 15|
| 19|
| 14|
| -1|
|  8|
| 10|
|  4|
| 13|
| -1|
| 17|
|  1|
|  3|
+---+

## History
Delta's most powerful feature is the ability to allow looking into history i.e., the changes that were made to the underlying Delta Table. The cell below shows how simple it is to inspect the history.

In [14]:
delta_table.history().show(20, 1000, False)

StatementMeta(small, 8, 16, Finished, Available)

+-------+-------------------+------+--------+---------+-------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|version|          timestamp|userId|userName|operation|                                                operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|                                                                                                                                                                                              operationMetrics|
+-------+-------------------+------+--------+---------+-------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+--------------------

## Read older versions of data using Time Travel

You can query previous snapshots of your Delta Lake table by using a feature called Time Travel. If you want to access the data that you overwrote, you can query a snapshot of the table before you overwrote the first set of data using the versionAsOf option.

Once you run the cell below, you should see the first set of data, from before you overwrote it. Time Travel is an extremely powerful feature that takes advantage of the power of the Delta Lake transaction log to access data that is no longer in the table. Removing the version 0 option (or specifying version 1) would let you see the newer data again. For more information, see [Query an older snapshot of a table (time travel)](https://docs.delta.io/latest/delta-batch.html#deltatimetravel).

In [15]:
df = spark.read.format("delta").option("versionAsOf", 0).load(delta_table_path)
df.show()

StatementMeta(small, 8, 17, Finished, Available)

+---+
| id|
+---+
|  4|
|  0|
|  1|
|  3|
|  2|
+---+

## Write a stream of data to a table

You can also write to a Delta Lake table using Spark's Structured Streaming. The Delta Lake transaction log guarantees exactly-once processing, even when there are other streams or batch queries running concurrently against the table. By default, streams run in append mode, which adds new records to the table.

For more information about Delta Lake integration with Structured Streaming, see [Table Streaming Reads and Writes](https://docs.delta.io/latest/delta-streaming.html).

In the cells below, here's what we are doing:

1. *Cell 28* Setup a simple Spark Structured Streaming job to generate a sequence and make the job write into our Delta Table
2. *Cell 30* Show the newly appended data
3. *Cell 31* Inspect history
4. *Cell 32* Stop the structured streaming job
5. *Cell 33* Inspect history <-- You'll notice appends have stopped

In [16]:
streaming_df = spark.readStream.format("rate").load()
stream = streaming_df\
    .selectExpr("value as id")\
    .writeStream\
    .format("delta")\
    .option("checkpointLocation", "/tmp/checkpoint-{0}".format(session_id))\
    .start(delta_table_path)

StatementMeta(small, 8, 18, Finished, Available)

## Read a stream of changes from a table

While the stream is writing to the Delta Lake table, you can also read from that table as streaming source. For example, you can start another streaming query that prints all the changes made to the Delta Lake table.

In [17]:
delta_table.toDF().sort(col("id").desc()).show(100)

StatementMeta(small, 8, 19, Finished, Available)

+---+
| id|
+---+
| 19|
| 18|
| 17|
| 16|
| 15|
| 14|
| 13|
| 12|
| 11|
| 10|
|  8|
|  6|
|  4|
|  3|
|  2|
|  1|
|  0|
| -1|
| -1|
| -1|
+---+

In [18]:
delta_table.history().drop("userId", "userName", "job", "notebook", "clusterId", "isolationLevel", "isBlindAppend").show(20, 1000, False)

StatementMeta(small, 8, 20, Finished, Available)

+-------+-------------------+----------------+-------------------------------------------------------------------------------------+-----------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|version|          timestamp|       operation|                                                                  operationParameters|readVersion|                                                                                                                                                                                              operationMetrics|
+-------+-------------------+----------------+-------------------------------------------------------------------------------------+-----------+--------------------------------------------------------------------------------------------------------------------------------------------------------

In [19]:
stream.stop()

StatementMeta(small, 8, 21, Finished, Available)

In [20]:
delta_table.history().drop("userId", "userName", "job", "notebook", "clusterId", "isolationLevel", "isBlindAppend").show(100, 1000, False)

StatementMeta(small, 8, 22, Finished, Available)

+-------+-------------------+----------------+-------------------------------------------------------------------------------------+-----------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|version|          timestamp|       operation|                                                                  operationParameters|readVersion|                                                                                                                                                                                              operationMetrics|
+-------+-------------------+----------------+-------------------------------------------------------------------------------------+-----------+--------------------------------------------------------------------------------------------------------------------------------------------------------

## Compaction

If a Delta Table is growing too large, you can compact it by repartitioning into a smaller number of files.

The option `dataChange = false` is an optimization that tells Delta Lake to do the repartition without marking the underlying data as "modified". This ensures that any other concurrent operations (such as streaming reads/writes) aren't negatively impacted.


In [21]:
partition_count = 2

spark.read\
    .format("delta")\
    .load(delta_table_path)\
    .repartition(partition_count)\
    .write.option("dataChange", "false")\
    .format("delta")\
    .mode("overwrite")\
    .save(delta_table_path)    

StatementMeta(small, 8, 23, Finished, Available)

## Convert Parquet to Delta
You can do an in-place conversion from the Parquet format to Delta.

In [22]:
parquet_path = "/parquet/parquet-table-{0}".format(session_id)

data = spark.range(0,5)
data.write.parquet(parquet_path)

# Confirm that the data isn't in the Delta format
DeltaTable.isDeltaTable(spark, parquet_path)

StatementMeta(small, 8, 24, Finished, Available)

False

In [23]:
DeltaTable.convertToDelta(spark, "parquet.`{0}`".format(parquet_path))

# Confirm that the converted data is now in the Delta format
DeltaTable.isDeltaTable(spark, parquet_path)

StatementMeta(small, 8, 25, Finished, Available)

True

## SQL Support
Delta supports table utility commands through SQL.  You can use SQL to:
* Get a DeltaTable's history
* Vacuum a DeltaTable
* Convert a Parquet file to Delta


In [24]:
spark.sql("DESCRIBE HISTORY delta.`{0}`".format(delta_table_path)).show()

StatementMeta(small, 8, 26, Finished, Available)

+-------+-------------------+------+--------+----------------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+
|version|          timestamp|userId|userName|       operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|
+-------+-------------------+------+--------+----------------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+
|      8|2021-02-23 05:33:11|  null|    null|           WRITE|[mode -> Overwrit...|null|    null|     null|          7|          null|        false|[numFiles -> 2, n...|
|      7|2021-02-23 05:33:02|  null|    null|STREAMING UPDATE|[outputMode -> Ap...|null|    null|     null|          6|          null|         true|[numRemovedFiles ...|
|      6|2021-02-23 05:32:59|  null|    null|STREAMING UPDATE|[outputMode -> Ap...|null|    null|     null|          5|          null|         true|[n

In [25]:
spark.sql("VACUUM delta.`{0}`".format(delta_table_path)).show()

StatementMeta(small, 8, 27, Finished, Available)

+--------------------+
|                path|
+--------------------+
|abfss://zhaotest@...|
+--------------------+

In [26]:
parquet_id = random.randint(0,1000)
parquet_path = "/parquet/parquet-table-{0}-{1}".format(session_id, parquet_path)

data = spark.range(0,5)
data.write.parquet(parquet_path)

# Confirm that the data isn't in the Delta format
DeltaTable.isDeltaTable(spark, parquet_path)

# Use SQL to convert the parquet table to Delta
spark.sql("CONVERT TO DELTA parquet.`{0}`".format(parquet_path))

DeltaTable.isDeltaTable(spark, parquet_path)

StatementMeta(small, 8, 28, Finished, Available)

True